## Max Precipitation by Year

The following code computes and shows the maximum precipitation event by year for each reservoir.

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.widgets import Slider, Button, RadioButtons
import pandas as pd
import datetime

# Reservoirs with discharge data
reservoirs = [
    'castaic',
    'del_valle',
    'feather',
    'pyramid',
    'san_luis',
    'thermalito']

noaa_sites = {
    'castaic': 'USC00048014',
    'del_valle': 'USC00045933',
    'feather': 'USC00047195',
    'pyramid': 'USW00023187',
    'san_luis': 'USC00045119',
    'thermalito': 'USC00046521',
}

def load_noaa_data(reservoir):
    if not reservoir in reservoirs:
        print('Reservoir not found')
        return
    
    filename = '../data/'+reservoir+'/noaa_'+noaa_sites[reservoir]+".csv"
    
    noaa_data = pd.read_csv(filename,index_col="DATE",low_memory=False)
    precipitation = precipitation = noaa_data.loc[:,"PRCP"]

    dates = pd.to_datetime(precipitation.index.get_values())
    min_year = dates[0].year
    max_year = dates[-1].year
    
    max_events = []
    for year in range(min_year, max_year):
        max_event = 0.
        for event in precipitation.loc[f'{year}-01-01':f'{year}-12-31']:
            if not pd.isna(event) and event > max_event:
                max_event = event
        max_events.append(max_event)
        
    precipitation = pd.Series(max_events, index=pd.date_range(datetime.date(min_year,1,1), freq='1Y', periods=(max_year-min_year)))
    dates = pd.to_datetime(precipitation.index.get_values())
        
    return dates, precipitation

default_reservoir = 'feather'
reservoir = default_reservoir

# Get maximum precipitation by year
dates, precipitation = load_noaa_data(reservoir)

min_date = dates[0]
max_date = dates[-1]

fig = plt.figure(figsize=(10, 6), dpi=80)
ax = fig.gca()

# Shrink the plot by 25% towards upper-right corner
plt.subplots_adjust(left=0.25, bottom=0.25)
l, = plt.plot(dates, precipitation.values, linewidth=2, color='red')

plt.xlabel('Date')
plt.ylabel('Max Watershed Precipitation (in)')
ax.xaxis.set_major_formatter(DateFormatter('%Y'))
ax.set_title(f'Max Precipitation\n{reservoir}')

axcolor = 'lightgoldenrodyellow'
axyear_start = plt.axes([0.25, 0.125, 0.65, 0.03], facecolor=axcolor)
axyear_end = plt.axes([0.25, 0.085, 0.65, 0.03], facecolor=axcolor)

syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')

def find_index(dates, year):
    index = 0
    for date in dates:
        if date.year == int(year):
            return index
        index += 1
    return -1

def update(val):
    year_start = syear_start.val
    year_end = syear_end.val
    if not year_start < year_end:
        return
    s_index = find_index(dates, year_start)
    e_index = find_index(dates, year_end + 1)
    l.set_xdata(dates[s_index:e_index])
    l.set_ydata(precipitation[s_index:e_index])
    ax.relim()
    ax.autoscale_view()
    plt.draw()
syear_start.on_changed(update)
syear_end.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

def reset(event):
    global syear_start, syear_end, reservoir_radio, min_date, max_date, dates, precipitation, reservoir
    reservoir = default_reservoir
    # Load new data
    dates, precipitation = load_noaa_data(reservoir)
    min_date = dates[0]
    max_date = dates[-1]
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(precipitation)
    ax.set_title(f'Max Precipitation\n{reservoir}')
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    # Reset radio buttons
    rax.clear()
    reservoir_radio = RadioButtons(rax, reservoirs, active=reservoirs.index(default_reservoir))
    reservoir_radio.on_clicked(set_reservoir)
    fig.canvas.flush_events()
button.on_clicked(reset)

rax = plt.axes([0.025, 0.6, 0.15, 0.3], facecolor=axcolor)
reservoir_radio = RadioButtons(rax, reservoirs, active=reservoirs.index(default_reservoir))

def set_reservoir(label):
    global syear_start, syear_end, station_radio, min_date, max_date, dates, precipitation, reservoir
    reservoir = label
    # Load new data
    dates, precipitation = load_noaa_data(reservoir)
    min_date = dates[0]
    max_date = dates[-1]
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(precipitation)
    ax.set_title(f'Max Precipitation\n{reservoir}')
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    fig.canvas.flush_events()
reservoir_radio.on_clicked(set_reservoir)

plt.show()

FigureCanvasNbAgg()

## Summary statistics

The following code prints the summary statistics for each reservoir.

In [3]:
for reservoir in reservoirs:
    dates, precipitation = load_noaa_data(reservoir)
    print(reservoir)
    print(precipitation.describe())
    print('\n')

castaic
count    100.000000
mean       2.416200
std        1.091511
min        0.680000
25%        1.625000
50%        2.200000
75%        3.062500
max        5.800000
dtype: float64


del_valle
count    70.000000
mean      1.958571
std       0.991400
min       0.790000
25%       1.465000
50%       1.775000
75%       2.147500
max       6.870000
dtype: float64


feather
count    123.000000
mean       3.269431
std        1.144607
min        0.000000
25%        2.575000
50%        3.230000
75%        3.780000
max        9.000000
dtype: float64


pyramid
count    70.000000
mean      2.192857
std       1.297365
min       0.430000
25%       1.160000
50%       1.930000
75%       2.950000
max       6.320000
dtype: float64


san_luis
count    86.000000
mean      0.980581
std       0.474176
min       0.000000
25%       0.717500
50%       0.920000
75%       1.277500
max       2.410000
dtype: float64


thermalito
count    26.000000
mean      2.247308
std       0.951406
min       1.340000
25%      